In [1]:
!pip install adamp

  Created wheel for adamp: filename=adamp-0.3.0-py3-none-any.whl size=5998 sha256=12847a274168274e6167333bac3650b0be981526a9bf77a72d48b21b0ccebfcc
  Stored in directory: /root/.cache/pip/wheels/bb/95/21/ced2d2cb9944e3a72e58fece7958973eed3fd8d0aeb6e2e450
Successfully built adamp


In [2]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 7.9 MB/s 
     |████████████████████████████████| 6.5 MB 77.9 MB/s 
     |████████████████████████████████| 895 kB 82.4 MB/s 
     |████████████████████████████████| 596 kB 72.5 MB/s 
     |████████████████████████████████| 67 kB 4.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
import os

import transformers
from transformers import AutoTokenizer, AdamW, RobertaForSequenceClassification
from transformers import get_linear_schedule_with_warmup

import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset

from tqdm.notebook import tqdm, tqdm_notebook

import random
import torch.backends.cudnn as cudnn

from sklearn.model_selection import StratifiedKFold

from adamp import AdamP

In [5]:
train_1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/nli/train_data.csv')
train_2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/nli/plus_data.csv')
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dacon/nli/test_data.csv")
submission = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dacon/nli/sample_submission.csv")


In [6]:
train=pd.concat([train_1, train_2])

In [7]:
list1 = [(train['label']== "entailment"), (train['label']== "contradiction"), (train['label']== "neutral")]
choicelist1 = [0,1,2]
train['label']=np.select(list1, choicelist1)

train=train[['premise','hypothesis','label']]
test=test[['premise','hypothesis']]

In [8]:
class TRAINDataset(Dataset):
  
  def __init__(self, data):
    self.dataset = data
    self.tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

    print(self.dataset)
  
  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    row = self.dataset.iloc[idx, 0:3].values
    sentence1 = row[0]
    sentence2 = row[1]
    y = row[2]
    inputs = self.tokenizer(
        sentence1,
        sentence2,
        truncation=True,
        return_token_type_ids=False,
        pad_to_max_length=True,
        add_special_tokens=True,
        max_length=100
    )
    
    input_ids = torch.from_numpy(np.asarray(inputs['input_ids']))
    attention_mask = torch.from_numpy(np.asarray(inputs['attention_mask']))

    return input_ids, attention_mask, y

In [9]:
class TESTDataset(Dataset):
  
  def __init__(self, data):
    self.dataset = data
    self.tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

    print(self.dataset)
  
  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    row = self.dataset.iloc[idx, 0:2].values
    sentence1 = row[0]
    sentence2 = row[1]
    inputs = self.tokenizer(
        sentence1,
        sentence2,
        truncation=True,
        return_token_type_ids=False,
        pad_to_max_length=True,
        add_special_tokens=True,
        max_length=100
    )
    
    input_ids = torch.from_numpy(np.asarray(inputs['input_ids']))
    attention_mask = torch.from_numpy(np.asarray(inputs['attention_mask']))

    return input_ids, attention_mask

In [10]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [11]:
device = torch.device("cuda")

In [12]:
# 파라미터
epochs = 20
batch_size = 16

In [13]:
# 모델 학습 및 검증
def training(train_dataset,val_dataset, fold):
  best_acc = 0
  
  model = RobertaForSequenceClassification.from_pretrained("klue/roberta-large", num_labels=3).to(device)
  
  dataset_train = TRAINDataset(train_dataset)
  dataset_val = TRAINDataset(val_dataset)

  train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
  valid_loader = DataLoader(dataset_val, batch_size=batch_size, shuffle=False)

  optimizer = AdamP(model.parameters(), lr=1e-5, betas=(0.9, 0.999), weight_decay=1e-2)

  total_steps = len(train_loader) * epochs

  # 스케줄러
  scheduler = get_linear_schedule_with_warmup(optimizer, 
                                              num_warmup_steps = 0,
                                              num_training_steps = total_steps)

  for e in range(epochs):
    train_acc = 0.0
    valid_acc = 0.0
    model.train()
    for batch_id, (token_ids, attention_masks, label) in tqdm(enumerate(train_loader), total=len(train_loader)):
      optimizer.zero_grad()
      token_ids = token_ids.to(device)
      attention_masks = attention_masks.to(device)
      label = label.to(device)
      out = model(token_ids, attention_masks)[0]
      loss = F.cross_entropy(out, label)
      loss.backward()
      optimizer.step()
      scheduler.step()
      train_acc += calc_accuracy(out, label)

    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (token_ids, attention_masks, label) in tqdm(enumerate(valid_loader), total=len(valid_loader)):
      token_ids = token_ids.to(device)
      attention_masks = attention_masks.to(device)
      label = label.to(device)
      out = model(token_ids, attention_masks)[0]
      valid_acc += calc_accuracy(out, label)
    print("epoch {} valid acc {}".format(e+1, valid_acc / (batch_id+1)))
    torch.save(model, '/content/drive/MyDrive/Colab Notebooks/dacon/nli/model'+str(fold)+'.pt')

In [14]:
# 교차검증
def main():
    seed= 2021 # 재현성을 위한 시드값 고정
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = False  # type: ignore

    # kfold
    kfold=[]

    splitter = StratifiedKFold(n_splits=5, shuffle=True, random_state=2021)
    for train_idx, val_idx in splitter.split(train.iloc[:, 0:2],train.iloc[:, 2]):
        kfold.append((train.iloc[train_idx,:],train.iloc[val_idx,:]))

    for fold,(train_datasets, valid_datasets) in enumerate(kfold):
        print(f'fold{fold} 학습중...')
        training(train_dataset=train_datasets,val_dataset=valid_datasets,fold=fold)

In [ ]:
main() 

fold0 학습중...


Downloading:   0%|          | 0.00/547 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classif

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

...

  0%|          | 0/1400 [00:00<?, ?it/s]

epoch 9 train acc 0.9955803571428572


  0%|          | 0/350 [00:00<?, ?it/s]

epoch 9 valid acc 0.9025


  0%|          | 0/1400 [00:00<?, ?it/s]

In [15]:
# 예측 
def inference(model, dataset_test):
    test_dataset = TESTDataset(dataset_test)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    model.eval()
    output_pred = []
    with torch.no_grad():
      for batch_id, (token_ids, attention_masks) in tqdm(enumerate(test_loader), total=len(test_loader)):
        token_ids = token_ids.long().to(device)
        attention_masks = attention_masks.long().to(device)
        output=model(token_ids, attention_masks)[0]
        logits = torch.nn.functional.softmax(output, dim=1).detach().cpu().numpy()
        output_pred.extend(logits)
    return output_pred

In [16]:
label_dict = {"entailment" : 0, "contradiction" : 1, "neutral" : 2}

In [20]:
# 결과 도출
def inference_main():
  res = np.zeros((len(test),3)) 
  for i in range(5): 
    print(f'fold{i} 모델 추론중...')
    # load my model
    model = torch.load('/content/drive/MyDrive/Colab Notebooks/dacon/nli/model'+str(i)+'.pt')

    pred_answer = inference(model, test)

    res += np.array(pred_answer) / 5 

  ans= np.argmax(res, axis=-1)
  out = [list(label_dict.keys())[_] for _ in ans]
  submission["label"] = out

In [18]:
inference_main()

fold0 모델 추론중...


Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/734k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

                                                premise  \
0                             다만 조금 좁아서 케리어를 펼치기 불편합니다.   
1                            그리고 위치가 시먼역보다는 샤오난먼역에 가까워요   
2                     구구절절 설명하고 이해시키려는 노력이 큰 의미없이 다가온다.   
3                              몇 번을 다시봐도 볼 때마다 가슴이 저민다.   
4          8월 중에 입주신청을 하면 청년은 9월, 신혼부부는 10월부터 입주가 가능하다.   
...                                                 ...   
1661  또 작업자의 숙련도와 경험 향상, 전문성을 요구하는 난이도 높은 데이터 가공을 통해...   
1662                   결말을 보니 아무래도 이 영화는 2부가 계획된 듯 합니다.   
1663  사회적 거리 두기 상황에서 총리도 카페를 갔다가 자리가 없어서 퇴짜 맞은 일도 있을...   
1664                            로마에서 3박4일간 이곳에서 머물렀습니다.   
1665                             난 당신이 떠날때 길 하나도 못 건넜는데   

                                hypothesis  
0                    케리어를 펼치기에 공간이 충분했습니다.  
1               시먼역보다는 샤오난먼역에 먼저 도착할 수 있어요  
2        무엇인가 말을 많이 하기는 했지만 큰 의미가 있지는 않았다.  
3                           다시 봤을때는 무덤덤했다.  
4     8월 중에 입주신청을 하면 신혼부부는 9월 부터 입주가 가능하다.  
...                  

  0%|          | 0/105 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [19]:
submission.to_csv("FOLD5(20)_submission.csv", index = False)